In [ ]:
siflis_xls = pd.ExcelFile('siflis.xlsx')
syphilis_0 = pd.read_excel(siflis_xls, 'DADOS')
syphilis_1 = pd.read_excel(siflis_xls, 'DADOS CONTINUAÇÃO 1')
syphilis_2 = pd.read_excel(siflis_xls, 'DADOS CONTINUAÇÃO 2')
syphilis_3 = pd.read_excel(siflis_xls, 'DADOS CONTINUAÇÃO 3')
syphilis_4 = pd.read_excel(siflis_xls, 'DADOS CONTINUAÇÃO 4')
syphilis_5 = pd.read_excel(siflis_xls, 'DADOS CONTINUAÇÃO 5')

syphilis_0.columns = syphilis_0.iloc[0]
syphilis_1.columns = syphilis_1.iloc[0]
syphilis_2.columns = syphilis_2.iloc[0]
syphilis_3.columns = syphilis_3.iloc[0]
syphilis_4.columns = syphilis_4.iloc[0]
syphilis_5.columns = syphilis_5.iloc[0]

syphilis_0 = syphilis_0.drop(0)
syphilis_1 = syphilis_1.drop(0)
syphilis_2 = syphilis_2.drop(0)
syphilis_3 = syphilis_3.drop(0)
syphilis_4 = syphilis_4.drop(0)
syphilis_5 = syphilis_5.drop(0)

dados 1
Sífilis Primária
Sífilis Secundária
Sífilis Latente   // possivelmente o melhor contrafactual

dados 2
Tx Det 

dados 5
Sífilis Adquirida
Tx Det 
Sífilis Adquirida Homens

In [ ]:
def melt_df(df, column, group_name, year):
    cols_hep = ["Código", "Nome Município"]
    dict_rename_hep = {}

    for y in range(2013, year+1):
        cols_hep.append(f"{column}{y}")
        dict_rename_hep[f"{column}{y}"] = y

    df_pd = (df[cols_hep]
     .rename(columns=dict_rename_hep)
     .melt(id_vars=["Código", "Nome Município"], value_vars=range(2013, year+1),
            var_name="year", value_name="cases") )
    df_pd = df_pd.rename(columns={"Código":"city_code",
                                  "Nome Município":"city_name"})
    df_pd = df_pd.set_index("city_code")
    df_pd["group"] = group_name
    return df_pd

hepatites_b = melt_df(hep_dados, "Hepatite B ", "hepatites_B", 2019)
indicadores = pd.read_csv(f"{path}/indicadores.csv")
total_hiv = melt_df(indicadores, "Casos ", "total_hiv", 2022)

In [ ]:
hetero = hiv_df[["city_name", "city_code", "hetero_2013",
                 "hetero_2014", "hetero_2015", "hetero_2016", "hetero_2017", "hetero_2018",
                 "hetero_2019", "hetero_2020", "hetero_2021", "hetero_2022"]]

bi = hiv_df[["city_name", "city_code", "bissexual_2013",
                    "bissexual_2014", "bissexual_2015", "bissexual_2016", "bissexual_2017", 
                    "bissexual_2018", "bissexual_2019", "bissexual_2020", "bissexual_2021", 
                    "bissexual_2022"]]

homo = hiv_df[["city_name", "city_code", "homossexual_2013",
                "homossexual_2014", "homossexual_2015", "homossexual_2016", "homossexual_2017", 
                "homossexual_2018", "homossexual_2019", "homossexual_2020", "homossexual_2021", 
                "homossexual_2022"]]

hiv_male_cols = ["city_name", "city_code"]
[ hiv_male_cols.append( "total_cases_male_" + str(c) ) for c in range(2013, 2021) ]
hiv_male = hiv_df[hiv_male_cols]


new_keys_hetero = {key : key.replace("hetero_", "") for key in hiv_df.columns}
new_keys_bi = {key : key.replace("bissexual_", "") for key in hiv_df.columns}
new_keys_homo = {key : key.replace("homossexual_", "") for key in hiv_df.columns}
new_keys_male = {key : key.replace("total_cases_male_", "") for key in hiv_df.columns}

hetero = hetero.rename(columns=new_keys_hetero)#.set_index("city_code")
bi = bi.rename(columns=new_keys_bi)#.set_index("city_code")
homo = homo.rename(columns=new_keys_homo)#.set_index("city_code")
hiv_male = hiv_male.rename(columns=new_keys_male)

years = ["2013", "2014", "2015", "2016", "2017", "2018", "2019"]
hetero_pd = pd.melt(hetero, id_vars=["city_code", "city_name"], 
        value_vars=years, 
        var_name="year", value_name='cases', ignore_index=False
       ).sort_values("city_code")
homo_pd = pd.melt(homo, id_vars=["city_code", "city_name"], 
        value_vars=years, 
        var_name="year", value_name='cases', ignore_index=False
       ).sort_values("city_code")
bi_pd = pd.melt(bi, id_vars=["city_code", "city_name"], 
        value_vars=years, 
        var_name="year", value_name='cases', ignore_index=False
       ).sort_values("city_code")
hiv_male_pd = pd.melt(hiv_male, id_vars=["city_code", "city_name"], 
        value_vars=years, 
        var_name="year", value_name='cases', ignore_index=False
       ).sort_values("city_code")


hetero_pd["group"] = "HET"
homo_pd["group"] = "HOMO"
bi_pd["group"] = "BI"
hiv_male_pd["group"] = "MALE"

hsh_pd = bi_pd.copy()
hsh_pd["cases"] = bi_pd["cases"] + homo_pd["cases"]
hsh_pd["group"] = "HSH hiv"

#hetero_pd["year"] = pd.to_datetime(hetero_pd["year"], format='%Y')
hetero_pd=hetero_pd.set_index("city_code")
homo_pd=homo_pd.set_index("city_code")
bi_pd=bi_pd.set_index("city_code")
hsh_pd=hsh_pd.set_index("city_code")
#df = pd.concat([hetero_pd,homo_pd,bi_pd])
cases = pd.concat([total_hiv, hepatites_b])


cases["year"] = cases["year"].astype(int)
cases = cases.sort_values(["city_code", "group", "year"])
cases["region_type"] = "city"
cases = pd.concat([cases, hepatites_b])
cases.loc[cases.index < 100, "region_type"] = "state"
cases.loc[cases.index <= 10, "region_type"] = "region"
cases.loc[55, "region_type"] = "brazil"
cases = cases[cases.index != 0]

states = (cases
    .reset_index()
    .query("region_type == 'state' & year == 2018 & group == 'total_hiv' ")
    [["city_code", "city_name"]]
          .rename(columns={"city_code":"state_code",
                       "city_name":"state_name"}))

cities = (cases
    .query("region_type == 'city' & year == 2018 & group == 'total_hiv' ")
).reset_index()
cities["state_code"] = cities["city_code"].astype(str).str.slice(0, 2).astype(int)

city_df = cities.merge(states, 
             left_on="state_code",
             right_on="state_code",
             how="inner")
city_df.to_csv("city_state.csv", mode="w")

#### treatment assignment

In [ ]:
import pandas as pd
prep = pd.DataFrame()
for year in range(2018, 2023):
    df = pd.read_csv(f"data/prep{year}.csv")
    df["year"] = year
    prep = pd.concat([prep, df])
prep_data = prep.groupby(["city_code", "year"]).tail(1)

new_cases = (prep_data.set_index(["city_code","year"])
  .sort_values(["city_code","year"])
    .groupby("city_code")
    .agg( {"following":"diff"} )
    .reset_index()
    .rename(columns={"following":"new_users"}))
prep_data = prep_data.merge(new_cases, left_on=["city_code","year"],
                            right_on=["city_code","year"],how="left")

prep_data.loc[prep_data["year"] == 2018, "new_users"] = prep_data["following"]


hiv2018 = cases.query("year == 2018 ")[["code", "hiv_total"]]
hiv2019 = cases.query("year == 2019 ")[["code", "hiv_total"]]

prep2018 = prep_data.query("year == 2018")[["city_code", "following"]]
prep2019 = prep_data.query("year == 2019")[["city_code", "following"]]

prep2018 = prep2018.merge(hiv2018, left_on="city_code", right_on="code")
prep2019 = prep2019.merge(hiv2019, left_on="city_code", right_on="code")

prep2018["percent_treat_18"] = prep2018["following"]/prep2018["hiv_homo"]
prep2019["percent_treat_19"] = prep2019["following"]/prep2019["hiv_homo"]

filter_2018_20 = (prep2018["percent_treat_18"] >= 0.2)
prep2018.loc[filter_cases, "treat20_18"] = 1
filter_2018_50 = (prep2018["percent_treat_18"] >= 0.5)
prep2018.loc[filter_cases, "treat50_18"] = 1

filter_2019_20 = (prep2019["percent_treat_19"] >= 0.2)
prep2019.loc[filter_cases, "treat20_19"] = 1
filter_2019_50 = (prep2019["percent_treat_19"] >= 0.5)
prep2019.loc[filter_cases, "treat50_19"] = 1